In [ ]:
from warnings import filterwarnings

filterwarnings("ignore")

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.nonparametric.smoothers_lowess import lowess

from quant_invest_lab.data_provider import download_crypto_historical_data
from quant_invest_lab.backtest import ohlc_long_only_backtester, ohlc_short_only_backtester


In [ ]:
SYMBOL = "BTC-USDT"
TIMEFRAME = "12hour"
df = download_crypto_historical_data(SYMBOL, TIMEFRAME).iloc[-3300:]
df.dropna(inplace=True)
print(df.shape)
df.head()

In [ ]:
df["Close_denoised"] = lowess(
    df.Close.values, np.arange(0, len(df.Close)), frac=1 / 72
)[:, 1]

df["Close_denoised_signal"] = df["Close_denoised"].diff().fillna(1.0).apply(np.sign)


In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(
        name="Raw data",
        x=df.index,
        y=df["Close"],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="Denoised data",
        x=df.index,
        y=df["Close_denoised"],
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="LOWESS filtering",
)

# Offline backtest


## Long

In [ ]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.Close_denoised_signal == 1 else False


def sell_func(row: pd.Series, prev_row: pd.Series, trading_days: pd.Series) -> bool:
    return True if row.Close_denoised_signal == -1 else False


# Backtest your strategy
ohlc_long_only_backtester(
    df=df,
    long_entry_function=buy_func,
    long_exit_function=sell_func,
    timeframe=TIMEFRAME,
    initial_equity=1000,
)

## Short

In [ ]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.Close_denoised_signal == -1 else False


def sell_func(row: pd.Series, prev_row: pd.Series, trading_days: pd.Series) -> bool:
    return True if row.Close_denoised_signal == 1 else False


# Backtest your strategy
ohlc_short_only_backtester(
    df=df,
    short_entry_function=buy_func,
    short_exit_function=sell_func,
    timeframe=TIMEFRAME,
    initial_equity=1000,
)

# Online backtest


In [ ]:
df = download_crypto_historical_data(SYMBOL, TIMEFRAME).iloc[-3300:]


def online_denoise(
    closes: pd.Series,
    frac: float = 1 / 72,
    with_memory: bool = False,
    series_name: str = "Close",
):
    if with_memory is False:
        return np.sign(
            np.diff(
                lowess(closes.to_numpy(), np.arange(0, len(closes)), frac=frac)[:, 1]
            )[-1]
        )
    else:
        full_closes = df.loc[: closes.index[-1]][series_name]
        return np.sign(
            np.diff(
                lowess(
                    full_closes.to_numpy(), np.arange(0, len(full_closes)), frac=frac
                )[:, 1]
            )[-1]
        )


df["Close_denoised_signal"] = df.Close.rolling(300).apply(online_denoise, args=(1 / 72, False,"Close"))

df.dropna(inplace=True)
df.head()

In [ ]:
def buy_func(row: pd.Series, prev_row: pd.Series) -> bool:
    return True if row.Close_denoised_signal == 1 else False


def sell_func(row: pd.Series, prev_row: pd.Series, trading_days: int) -> bool:
    return True if row.Close_denoised_signal == -1 else False


# Backtest your strategy
ohlc_long_only_backtester(
    df=df,
    long_entry_function=buy_func,
    long_exit_function=sell_func,
    timeframe=TIMEFRAME,
    initial_equity=1000,
)